In [16]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [17]:
# %% [1] Imports and Basic Setup
!pip install pandas

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Adjust the path to your data
data_path = "../../data/round_2_data"


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [18]:
# %% [2] Load CSVs
# columns delimited by ';'
prices_day1 = pd.read_csv(f"{data_path}/prices_round_2_day_-1.csv", delimiter=';')  # day=-1
prices_day2 = pd.read_csv(f"{data_path}/prices_round_2_day_0.csv",  delimiter=';')  # day=0
prices_day3 = pd.read_csv(f"{data_path}/prices_round_2_day_1.csv",  delimiter=';')  # day=1

# We'll just load them all at once for Croissants / Jams
prices_all_croissants = pd.concat([
    prices_day1[prices_day1['product'] == 'CROISSANTS'],
    prices_day2[prices_day2['product'] == 'CROISSANTS'],
    prices_day3[prices_day3['product'] == 'CROISSANTS']
], ignore_index=True)

prices_all_jams = pd.concat([
    prices_day1[prices_day1['product'] == 'JAMS'],
    prices_day2[prices_day2['product'] == 'JAMS'],
    prices_day3[prices_day3['product'] == 'JAMS']
], ignore_index=True)

print("All Croissants shape:", prices_all_croissants.shape)
print("All Jams shape:", prices_all_jams.shape)

All Croissants shape: (30000, 17)
All Jams shape: (30000, 17)


In [19]:
# %% [3] Merge Croissants & Jams into One DF for ALL days
# We'll rename 'timestamp' => 'timestamp_croiss' and 'mid_price' => 'mid_price_croiss' or 'mid_price_jam'.

croiss = prices_all_croissants[['timestamp','mid_price']].rename(
    columns={'timestamp':'timestamp_croiss','mid_price':'mid_price_croiss'}
)
jams = prices_all_jams[['timestamp','mid_price']].rename(
    columns={'timestamp':'timestamp_croiss','mid_price':'mid_price_jam'}
)

df_all = pd.merge(croiss, jams, on='timestamp_croiss', how='inner')
print("Merged DataFrame for all days:\n", df_all.head())

# Sort by timestamp
df_sorted = df_all.sort_values(by="timestamp_croiss").reset_index(drop=True)

# Quick no-lag correlation
corr_no_lag = df_sorted["mid_price_croiss"].corr(df_sorted["mid_price_jam"])
print(f"\nNo-lag correlation (all days) = {corr_no_lag:.4f}")

Merged DataFrame for all days:
    timestamp_croiss  mid_price_croiss  mid_price_jam
0                 0            4304.5         6670.5
1                 0            4304.5         6632.0
2                 0            4304.5         6542.0
3                 0            4321.5         6670.5
4                 0            4321.5         6632.0

No-lag correlation (all days) = 0.0428


In [20]:
# %% [4] Define find_best_lag and run
def find_best_lag(df, col_croiss='mid_price_croiss', col_jam='mid_price_jam',
                  lag_min=-5000, lag_max=5000, lag_step=100):
    """
    Finds the lag that yields the highest correlation.
      df: DataFrame containing 'col_croiss' and 'col_jam'
      lag_min, lag_max: range of lags
      lag_step: increments between lags
    Returns: (best_lag, best_corr)
    
    Negative best_lag => Croissants lead Jam by |best_lag| ticks.
    Positive best_lag => Croissants lag Jam by best_lag ticks.
    """
    A = df[col_croiss].values
    B = df[col_jam].values

    best_corr = -999.0
    best_lag = 0

    for lag in range(lag_min, lag_max+1, lag_step):
        # Shift Croissant by 'lag'
        A_shifted = np.roll(A, lag)

        # Trim wrap-around portion
        if lag < 0:
            valid_slice = slice(0, len(A)+lag)
        else:
            valid_slice = slice(lag, len(A))

        A_valid = A_shifted[valid_slice]
        B_valid = B[valid_slice]

        if len(A_valid) > 2:
            corr_val = np.corrcoef(A_valid, B_valid)[0, 1]
            if corr_val > best_corr:
                best_corr = corr_val
                best_lag = lag

    return best_lag, best_corr

# Run the lag finder
best_lag, best_corr = find_best_lag(df_sorted,
    col_croiss="mid_price_croiss",
    col_jam="mid_price_jam",
    lag_min=-100000,
    lag_max=100000,
    lag_step=100
)

print(f"\nBest correlation (all days) = {best_corr:.4f} at lag = {best_lag}")
if best_lag < 0:
    print(f"Croissants lead Jam by {abs(best_lag)} ticks.")
elif best_lag > 0:
    print(f"Croissants lag Jam by {best_lag} ticks.")
else:
    print("No lead/lag advantage at lag=0.")


Best correlation (all days) = 0.3429 at lag = -86300
Croissants lead Jam by 86300 ticks.
